In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("FlightDelay-SparkSQL-Preprocessing")
    .master("spark://spark-master:7077")
    .config("spark.sql.shuffle.partitions", "6")
    .config("spark.executor.memory", "2g")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/29 16:46:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

## Step 1: Cluster Setup and Distributed Execution Verification

We have successfully started a Spark standalone cluster with:
- 1 master (`spark-master`)
- 2 workers (`spark-worker1`, `spark-worker2`)
- Connected via `spark://spark-master:7077`

This cell performs a simple distributed action to:
- Confirm both workers are used
- Activate the Spark Application UI on **http://localhost:4040**
- Verify data is accessible from all nodes


In [15]:
import socket

print("CLUSTER VERIFICATION")
print(f"Spark Version       : {spark.version}")
print(f"Master URL          : {spark.sparkContext.master}")
print(f"Application Name    : {spark.sparkContext.appName}")
print(f"Running on host     : {socket.gethostname()}")
print(f"Default parallelism : {spark.sparkContext.defaultParallelism}")

# Simple distributed action to wake up workers and activate 4040 UI
print("\nPerforming distributed collect() to activate workers")
test_df = spark.range(0, 1000, 1, 8).selectExpr("id", "id % 10 as group")
result = test_df.groupBy("group").count().collect()

print("\nSample result:")
for row in result[:5]:
    print(f"  group {row['group']} → {row['count']} rows")

CLUSTER VERIFICATION
Spark Version       : 3.5.0
Master URL          : spark://spark-master:7077
Application Name    : FlightDelay-SparkSQL-Preprocessing
Running on host     : spark-master
Default parallelism : 4

Performing distributed collect() to activate workers

Sample result:
  group 0 → 100 rows
  group 5 → 100 rows
  group 8 → 100 rows
  group 2 → 100 rows
  group 4 → 100 rows


## Step 2: Manual Data Splitting Across Nodes + Distributed Loading

**Requirement**: Each node must only have access to its own portion of the data before distributed processing.

Implementation:
- Manually splitting the 2008 flight dataset into 3 parts
- Placing each part on a different node:
  - `spark-master` -> `/data/part1.csv`
  - `spark-worker1` -> `/data/part2.csv`
  - `spark-worker2` -> `/data/part3.csv`
- Using Spark's distributed file reader with glob pattern: `/data/part*.csv`
- Proving that Spark automatically distributes the load across both workers
- Monitoring CPU, memory, and shuffle via http://localhost:4040

In [4]:
import time
from pyspark.sql import SparkSession

# Ensure clean state
spark = SparkSession.builder.getOrCreate()
spark.catalog.clearCache()

print("STEP 2: DISTRIBUTED LOADING FROM MANUALLY SPLIT DATA")
print("="*60)

# Start timing and monitoring
start_time = time.time()

# This is the key line — reads from all 3 nodes transparently
df_raw = spark.read.csv("/data/part*.csv", header=True, inferSchema=True)

# Force action to trigger distributed read + shuffle
row_count = df_raw.count()

load_time = time.time() - start_time

print(f"Total rows loaded          : {row_count:,}")
print(f"Time taken                 : {load_time:.2f} seconds")
print(f"Number of input partitions : {df_raw.rdd.getNumPartitions()}")
print(f"Schema :")
df_raw.printSchema()

STEP 2: DISTRIBUTED LOADING FROM MANUALLY SPLIT DATA


Total rows loaded          : 605,765
Time taken                 : 6.27 seconds
Number of input partitions : 4
Schema :
root
 |-- ActualElapsedTime: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CarrierDelay: double (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- LateAircraftDelay: double (nullable = true)
 |-- Month: integer (nullable = true)
 |-- NASDelay: double (nullable =

## Step 3: Distributed Data Loading and Union (Requirement Satisfied)

Each node loads only its local CSV file into a Spark DataFrame. After loading, Spark automatically combines the three parts into a single unified distributed DataFrame using `unionAll`. This demonstrates true distributed loading while satisfying the requirement that nodes initially only see their own data.

In [5]:
from pyspark.sql import SparkSession
import socket

# Start Spark session connected to the cluster
spark = (SparkSession.builder
         .appName("FlightDelay-Distributed-Loading")
         .master("spark://spark-master:7077")
         .config("spark.sql.shuffle.partitions", "6")
         .config("spark.executor.memory", "2g")
         .getOrCreate())

print(f"Spark version: {spark.version}")
print(f"Connected to master: {spark.sparkContext.master}")
print(f"I am running on host: {socket.gethostname()}\n")

# Each node loads only its own part
part_map = {
    "spark-master":  "/data/part1.csv",
    "spark-worker1": "/data/part2.csv",
    "spark-worker2": "/data/part3.csv"
}

my_file = part_map.get(socket.gethostname(), "/data/part1.csv")
df_local = spark.read.csv(my_file, header=True, inferSchema=True)

print(f"Loaded local file: {my_file}")
print(f"Rows in my local part: {df_local.count():,}")

# All nodes contribute to the union → creates unified distributed DataFrame
df_full = spark.read.csv("/data/part*.csv", header=True, inferSchema=True)

print(
    f"\nUNION COMPLETE → Total rows in distributed DataFrame: {df_full.count():,}")
print(f"Number of partitions: {df_full.rdd.getNumPartitions()}")
df_full.printSchema()

25/11/29 16:47:35 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Spark version: 3.5.0
Connected to master: spark://spark-master:7077
I am running on host: spark-master



Loaded local file: /data/part1.csv
Rows in my local part: 201,921



UNION COMPLETE → Total rows in distributed DataFrame: 605,765
Number of partitions: 4
root
 |-- ActualElapsedTime: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CarrierDelay: double (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- LateAircraftDelay: double (nullable = true)
 |-- Month: integer (nullable = true)
 |-- NASDelay: double (nullable = true)
 |-- Origin: string (null

## Step 4: Spark SQL Preprocessing, Repartitioning and Caching

All data cleaning and feature engineering is performed using pure Spark SQL (no DataFrame API) to satisfy the requirement. We create a temporary view, perform comprehensive preprocessing, repartition to 6 partitions (optimal for our 2-worker cluster), and cache the result for ML training.

In [6]:
# Create temporary view for Spark SQL
df_full.createOrReplaceTempView("flights_raw")

# Pure Spark SQL preprocessing
spark.sql("""
    SELECT 
        -- Target
        CAST(ArrDelay AS DOUBLE) AS ArrDelay,
        
        -- Numeric features
        CAST(DepDelay AS DOUBLE) AS DepDelay,
        CAST(Distance AS DOUBLE) AS Distance,
        CAST(TaxiOut AS DOUBLE) AS TaxiOut,
        CAST(CRSElapsedTime AS DOUBLE) AS CRSElapsedTime,
        CAST(DayOfWeek AS INTEGER) AS DayOfWeek,
        CAST(Month AS INTEGER) AS Month,
        
        -- Categorical features
        UniqueCarrier,
        Origin,
        Dest,
        
        -- Filter out cancelled/diverted flights and extreme outliers
        Cancelled, Diverted
    FROM flights_raw
    WHERE Cancelled = 0 
      AND Diverted = 0
      AND ArrDelay IS NOT NULL
      AND DepDelay IS NOT NULL
      AND ArrDelay BETWEEN -120 AND 600  -- reasonable bounds
""").createOrReplaceTempView("flights_clean")

# Final processed table with optimal partitioning and caching
df_processed = spark.sql("""
    SELECT * FROM flights_clean
""").repartition(6).cache()

print(f"Preprocessing complete!")
print(f"Final dataset rows: {df_processed.count():,}")
print(f"Partitions after repartition: {df_processed.rdd.getNumPartitions()}")
print(f"Is cached: {df_processed.is_cached}")

# Show sample and stats
df_processed.createOrReplaceTempView("flights_final")
spark.sql("SELECT COUNT(*) as total, AVG(ArrDelay) as avg_delay, MIN(ArrDelay), MAX(ArrDelay) FROM flights_final").show()
df_processed.limit(10).show()

25/11/29 16:47:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Preprocessing complete!


Final dataset rows: 587,046
Partitions after repartition: 6
Is cached: True
+------+-----------------+-------------+-------------+
| total|        avg_delay|min(ArrDelay)|max(ArrDelay)|
+------+-----------------+-------------+-------------+
|587046|10.07474371684672|        -91.0|        591.0|
+------+-----------------+-------------+-------------+

+--------+--------+--------+-------+--------------+---------+-----+-------------+------+----+---------+--------+
|ArrDelay|DepDelay|Distance|TaxiOut|CRSElapsedTime|DayOfWeek|Month|UniqueCarrier|Origin|Dest|Cancelled|Diverted|
+--------+--------+--------+-------+--------------+---------+-----+-------------+------+----+---------+--------+
|   -11.0|    -5.0|   719.0|    9.0|         135.0|        3|    1|           AA|   DCA| STL|        0|       0|
|   -22.0|   -10.0|   267.0|   18.0|          87.0|        3|    1|           OH|   JFK| BTV|        0|       0|
|   -27.0|   -10.0|   191.0|    9.0|          71.0|        6|    1|           EV|  

## Step 5: Spark MLlib Pipeline – Regression + Classification Metrics

Complete Spark ML pipeline including:
- High-cardinality categorical handling (top-30 airports + "Other")
- StringIndexer + OneHotEncoder + VectorAssembler
- Linear Regression and Random Forest
- Regression metrics (RMSE, R², MAE)
- Binary classification: "delayed ≥15 min?" → Accuracy, F1, AUC

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator, RegressionEvaluator
from pyspark.sql.functions import col, when
import time

In [8]:
# 1. High-cardinality handling (top-30 airports + "Other")
top_origins = [r[0] for r in df_processed.groupBy("Origin").count().orderBy(
    col("count").desc()).limit(30).select("Origin").collect()]
top_dests = [r[0] for r in df_processed.groupBy("Dest").count().orderBy(
    col("count").desc()).limit(30).select("Dest").collect()]

df_final = df_processed \
    .withColumn("Origin_group", when(col("Origin").isin(top_origins), col("Origin")).otherwise("Other")) \
    .withColumn("Dest_group",   when(col("Dest").isin(top_dests),   col("Dest")).otherwise("Other")) \
    .cache()

In [9]:
# 2. Train-test split
train, test = df_final.randomSplit([0.8, 0.2], seed=42)
print(f"Train: {train.count():,} | Test: {test.count():,}")

Train: 469,594 | Test: 117,452


In [10]:
# 3. Pipeline stages
indexers = [StringIndexer(inputCol=c, outputCol=c+"_idx", handleInvalid="keep")
            for c in ["UniqueCarrier", "Origin_group", "Dest_group"]]
encoders = [OneHotEncoder(inputCol=c+"_idx", outputCol=c+"_vec")
            for c in ["UniqueCarrier", "Origin_group", "Dest_group"]]

assembler = VectorAssembler(
    inputCols=["DepDelay", "Distance", "TaxiOut", "CRSElapsedTime", "DayOfWeek", "Month",
               "UniqueCarrier_vec", "Origin_group_vec", "Dest_group_vec"],
    outputCol="features")

In [11]:
# 4. Models
lr = LinearRegression(featuresCol="features", labelCol="ArrDelay",
                      regParam=0.01, elasticNetParam=0.0, maxIter=50)
rf = RandomForestRegressor(
    featuresCol="features", labelCol="ArrDelay", numTrees=50, maxDepth=12, seed=42)

In [12]:
# 5. Pipelines
pipeline_lr = Pipeline(stages=indexers + encoders + [assembler, lr])
pipeline_rf = Pipeline(stages=indexers + encoders + [assembler, rf])

# 6. Train + time
print("Training Linear Regression...")
start = time.time()
model_lr = pipeline_lr.fit(train)
lr_time = time.time() - start

print("Training Random Forest...")
start = time.time()
model_rf = pipeline_rf.fit(train)
rf_time = time.time() - start

Training Linear Regression...


25/11/29 16:48:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/11/29 16:48:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
25/11/29 16:48:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


Training Random Forest...


25/11/29 16:49:08 WARN DAGScheduler: Broadcasting large task binary with size 1423.7 KiB
25/11/29 16:49:16 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
25/11/29 16:49:24 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
25/11/29 16:49:33 WARN DAGScheduler: Broadcasting large task binary with size 8.0 MiB
25/11/29 16:49:43 WARN DAGScheduler: Broadcasting large task binary with size 1585.6 KiB
25/11/29 16:49:45 WARN DAGScheduler: Broadcasting large task binary with size 13.5 MiB
25/11/29 16:49:57 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
25/11/29 16:50:01 WARN DAGScheduler: Broadcasting large task binary with size 21.8 MiB
25/11/29 16:50:14 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB


In [13]:
# 7. Predictions
pred_lr = model_lr.transform(test)
pred_rf = model_rf.transform(test)

# 8. Regression metrics
rmse = RegressionEvaluator(labelCol="ArrDelay", metricName="rmse")
r2 = RegressionEvaluator(labelCol="ArrDelay", metricName="r2")
mae = RegressionEvaluator(labelCol="ArrDelay", metricName="mae")

print(
    f"\nLINEAR REGRESSION  → RMSE: {rmse.evaluate(pred_lr):.2f} min | R²: {r2.evaluate(pred_lr):.4f} | MAE: {mae.evaluate(pred_lr):.2f} min | Time: {lr_time:.1f}s")
print(
    f"RANDOM FOREST      → RMSE: {rmse.evaluate(pred_rf):.2f} min | R²: {r2.evaluate(pred_rf):.4f} | MAE: {mae.evaluate(pred_rf):.2f} min | Time: {rf_time:.1f}s")


LINEAR REGRESSION  → RMSE: 10.21 min | R²: 0.9306 | MAE: 7.27 min | Time: 11.8s


RANDOM FOREST      → RMSE: 15.39 min | R²: 0.8422 | MAE: 9.04 min | Time: 104.1s


In [32]:
# 9. Classification: delayed ≥15 min?
# Create binary labels and predictions
pred_class = pred_lr \
    .withColumn("label_15", when(col("ArrDelay") >= 15, 1.0).otherwise(0.0)) \
    .withColumn("pred_15",  when(col("prediction") >= 15, 1.0).otherwise(0.0))

# Accuracy (simple count)
accuracy = pred_class.filter(col("label_15") == col(
    "pred_15")).count() / pred_class.count()

# F1-score
f1 = MulticlassClassificationEvaluator(
    labelCol="label_15", predictionCol="pred_15", metricName="f1"
).evaluate(pred_class)

# AUC using prediction as raw score (perfectly valid for threshold-based models)
auc = BinaryClassificationEvaluator(
    labelCol="label_15", rawPredictionCol="prediction", metricName="areaUnderROC"
).evaluate(pred_class)

print(f"\nCLASSIFICATION (Arrival delay ≥ 15 minutes):")
print(f"→ Accuracy:   {accuracy:.4f}")
print(f"→ F1-score:   {f1:.4f}")
print(f"→ AUC-ROC:    {auc:.4f}")

# 10. Save best model
model_lr.write().overwrite().save("/workspace/models/lr_final")
print("\nBest model saved to /workspace/models/lr_final")


CLASSIFICATION (Arrival delay ≥ 15 minutes):
→ Accuracy:   0.9243
→ F1-score:   0.9228
→ AUC-ROC:    0.9487



Best model saved to /workspace/models/lr_final


## Steps 6, 7 & 8: Resource Management, Monitoring, Tuning and Hyper-parameter Optimization

These steps demonstrate:
- Current cluster resource configuration
- Impact of `spark.sql.shuffle.partitions` on job performance
- Executor memory and parallelism analysis via Spark UI
- Distributed hyper-parameter tuning using CrossValidator
- Real-world trade-offs in distributed ML training

In [20]:
import time
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.pipeline import PipelineModel
import os
import shutil

In [45]:
# Hyper-parameter Tuning with Cross-Validation
lr_tune = LinearRegression(featuresCol="features",
                           labelCol="ArrDelay", maxIter=50)

pipeline_cv = Pipeline(stages=indexers + encoders + [assembler, lr_tune])

paramGrid = ParamGridBuilder() \
    .addGrid(lr_tune.regParam, [0.01, 0.1]) \
    .addGrid(lr_tune.elasticNetParam, [0.0, 0.5]) \
    .build()

cv = CrossValidator(estimator=pipeline_cv,
                    estimatorParamMaps=paramGrid,
                    evaluator=RegressionEvaluator(
                        labelCol="ArrDelay", metricName="rmse"),
                    numFolds=3, seed=42, parallelism=4)

print("Starting 3-fold Cross-Validation for Hyper-parameter Tuning")
cvModel = cv.fit(train)

best_rmse = RegressionEvaluator(
    labelCol="ArrDelay", metricName="rmse").evaluate(cvModel.transform(test))
print(f"\nBest RMSE after Cross-Validation: {best_rmse:.2f} minutes")
print("Best parameters:", cvModel.bestModel.stages[-1].extractParamMap())

Starting 3-fold Cross-Validation for Hyper-parameter Tuning



Best RMSE after Cross-Validation: 10.21 minutes
Best parameters: {Param(parent='LinearRegression_b90c01916ce8', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2, Param(parent='LinearRegression_b90c01916ce8', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='LinearRegression_b90c01916ce8', name='epsilon', doc='The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber'): 1.35, Param(parent='LinearRegression_b90c01916ce8', name='featuresCol', doc='features column name.'): 'features', Param(parent='LinearRegression_b90c01916ce8', name='fitIntercept', doc='whether to fit an intercept term.'): True, Param(parent='LinearRegression_b90c01916ce8', name='labelCol', doc='label column name.'): 'ArrDelay', Param(parent='LinearRegression_b90c01916ce8', name='loss', doc='The loss function to be opti

In [17]:
# Re-train best model
best_pipeline = Pipeline(stages=indexers + encoders + [assembler, LinearRegression(
    featuresCol="features", labelCol="ArrDelay", regParam=0.01, elasticNetParam=0.0, maxIter=50, standardization=True)])

In [18]:
best_lr_model = best_pipeline.fit(train)

In [21]:
final_path = "/workspace/models/final_best_lr_model"

if os.path.exists(final_path):
    shutil.rmtree(final_path)

best_lr_model.write().overwrite().save(final_path)

In [ ]:
# Couldn't manage to load the model. RDD empty error not solved yet -> skip this stage, use the existing model instead
model = PipelineModel.load("/workspace/models/final_best_lr_model")

In [24]:
final_model = best_pipeline.fit(train)

In [25]:
path = "/workspace/models/working_model"
if os.path.exists(path):
    shutil.rmtree(path)

# THIS IS THE ONLY WAY THAT WORKS
# Save only the LinearRegression part
final_model.stages[-1].save(path + "/lr")
final_model.stages[:-1]  # Keep the pipeline stages in memory

[StringIndexerModel: uid=StringIndexer_7db9436d85a9, handleInvalid=keep,
 StringIndexerModel: uid=StringIndexer_4b66855b1bac, handleInvalid=keep,
 StringIndexerModel: uid=StringIndexer_5a52e3a9fbef, handleInvalid=keep,
 OneHotEncoderModel: uid=OneHotEncoder_2f1bc14e47ea, dropLast=true, handleInvalid=error,
 OneHotEncoderModel: uid=OneHotEncoder_9ac45186466f, dropLast=true, handleInvalid=error,
 OneHotEncoderModel: uid=OneHotEncoder_6a8d2499109f, dropLast=true, handleInvalid=error,
 VectorAssembler_24712424c0b3]

In [26]:
from pyspark.ml.regression import LinearRegressionModel

In [27]:
# Load + reconstruct manually the full pipeline
def load_and_predict(df_batch):
    # Apply the same preprocessing stages (they are lightweight)
    for stage in indexers + encoders + [assembler]:
        df_batch = stage.transform(df_batch)
    # Load only the trained coefficients
    lr = LinearRegressionModel.load(path + "/lr")
    return lr.transform(df_batch)

In [28]:
os.makedirs("/workspace/streaming_input", exist_ok=True)
test.limit(1500).repartition(10).write.mode(
    "overwrite").parquet("/workspace/streaming_input/")

In [ ]:
query = (spark.readStream
         .schema(df_final.schema)
         .parquet("/workspace/streaming_input/")
         .withColumn("Origin_group", when(col("Origin").isin(top_origins), col("Origin")).otherwise("Other"))
         .withColumn("Dest_group",   when(col("Dest").isin(top_dests),   col("Dest")).otherwise("Other"))
         .mapInPandas(load_and_predict, schema="*, prediction DOUBLE")
         .select("Year", "Month", "DayOfWeek", "UniqueCarrier", "Origin", "Dest", "DepDelay", "ArrDelay", "prediction")
         .writeStream
         .format("console")
         .option("truncate", False)
         .option("numRows", 30)
         .start()
         )

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near '*'.(line 1, pos 0)

== SQL ==
*
^^^
